## How To Perform a Ramsey inteferometry Experiment

### Prerequisites
This guide assumes you have a configured `DeviceSetup` as well as `Qubit` objects with assigned parameters. Please see these guides (add links) if you need to create your setup and qubits for the first time. However, you can also run this notebook "as is" using an emulated session. If you are just getting started with the LabOne Q Applications Library, please don't hesitate to reach out to us at info@zhinst.com.

### Background
In this how-to guide, you'll perform an experiment to find the qubit frequency and $T_2^*$ time of transmons using the `ramsey` experiment included in the LabOne Q Applications Library. 

In a Ramsey inteferometry experiment, you'll vary the wait time, $t_{wait}$, between the two $X_{90}$ operations, and give a parameter `detuning`, $\Delta f$ in unit of Hz. A phase $\phi_2(t_{wait})$ is calculated from $t_{wait}$ by using the parameter `detuning`:

$$\phi_2(t_{wait}) = 2\pi \Delta f \cdot t_{wait} $$

and added to the second $X_{90}$ operation. This way, the second $X_{90}$ operation is equivalent to a rotation around the $z$-axis by $\phi_2(t_{wait})$, followed by a rotation around the $x$-axis by $\pi/2$.

### Imports

You'll start by importing the Ramsey experiment from `laboneq_applications`, as well as `laboneq.simple` and a demo QPU and device setup to run in emulation mode.

In [ ]:
import numpy as np
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation
from laboneq.simple import *

from laboneq_applications.experiments import ramsey
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import demo_platform

### QPU and Device Setup

You'll generate six qubits with pre-defined parameters, as well as a `Device_Setup` consisting of a SHFQC+, HDAWG, and PQSC. If you already have your own `DeviceSetup` and qubits configured, you'll instead initialize the session using your setup.

In [ ]:
my_platform = demo_platform(6)

Then, you'll connect to the `Session`. Here we connect to an emulated one:

In [ ]:
session = Session(my_platform.setup)
session.connect(do_emulation=True)

### Running the Ramsey Workflow

You'll now make the experiment workflow and run:

In [ ]:
# our qubits live here in the demo setup:
qubits = my_platform.qpu.qubits

my_workflow = ramsey.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    delays=[np.linspace(0, 1e-6, 10), np.linspace(0, 1e-6, 10)],
    detunings=[1e6, 1e6],
)

my_results = my_workflow.run()

#### Output Simulation

You can also inspect the compiled experiment and plot the simulated output:

In [ ]:
compiled_ramsey = my_results.tasks["compile_experiment"].output
plot_simulation(compiled_ramsey, length=50e-6)

#### Inspecting the Source Code

You can inspect the source code of the `create_experiment` task defined in `ramsey` to see how the experiment pulse sequence is created:

In [ ]:
ramsey.create_experiment.src

### Changing the Options

We can give our Rabi experiment options. First, inspect what they currently are:

In [ ]:
my_new_opts = ramsey.experiment_workflow.options()
my_new_opts

Then provide new options. This time, you'll disable the calibration traces and change the counts.

In [ ]:
my_new_opts.count(2048)
my_new_opts.use_cal_traces(value=False)
my_new_opts

#### Run the workflow with updated options

Now, run the workflow with new options and inspect the simulated output. You'll notice that the calibration traces are no longer there.

In [ ]:
my_new_workflow = ramsey.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    delays=[np.linspace(0, 1e-6, 10), np.linspace(0, 1e-6, 10)],
    detunings=[1e6, 1e6],
    options=my_new_opts,
)

my_new_results = my_new_workflow.run()
new_compiled_ramsey = my_new_results.tasks["compile_experiment"].output
plot_simulation(new_compiled_ramsey, length=50e-6)

### Temporary settings

The qubit parameters are used to control the settings of pulses and instruments during the experiment. We can run the amplitude-Rabi experiment with different settings by passing it a copy of the qubits with modified parameters:

In [ ]:
# Make a copy of the qubits
temp_qubits = my_platform.qpu.copy_qubits()
# Change the length of the drive pulses
temp_qubits[0].parameters.ge_drive_length = 1000e-9
temp_qubits[1].parameters.ge_drive_length = 1000e-9

my_new_workflow = ramsey.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[temp_qubits[0], temp_qubits[1]],  # pass temporary qubits
    delays=[np.linspace(0, 1e-6, 10), np.linspace(0, 1e-6, 10)],
    detunings=[1e6, 1e6],
    options=my_new_opts,
)

my_new_results = my_new_workflow.run()
new_compiled_ramsey = my_new_results.tasks["compile_experiment"].output
plot_simulation(new_compiled_ramsey, length=10e-6)

Great! You've now run your Ramsey experiment. Check out these other experiments to keep characterizing your qubits: